In [38]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal, Annotated
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
import operator

In [39]:
generator_llm = ChatOpenAI(model='gpt-4o-mini')
evaluator_llm = ChatOpenAI(model='gpt-4o-mini')
optimizer_llm = ChatOpenAI(model="gpt-4o-mini")

In [40]:
#state
class TweetState(TypedDict):
    topic: str
    tweet: str
    evaluation: Literal["approved","needs_improvement"]
    feedback: str   
    iteration: int
    max_iteration: int

    tweet_history: Annotated[list[str], operator.add]
    feedback_history: Annotated[list[str], operator.add]
    

In [41]:

from pydantic import BaseModel, Field

class TweetEvaluation(BaseModel):
    evaluation: Literal["approved","needs_improvement"] = Field(..., description="Final evaluation result.")
    feedback: str = Field(..., description="feedback for the tweet.")


In [42]:
structured_evaluator_llm=evaluator_llm.with_structured_output(TweetEvaluation)

In [43]:
def generate_tweet_func(state:TweetState):
    messages = [
        SystemMessage(content="You are a funny and clever Twitter/C influencer."),
        HumanMessage(content=f"""
Write a short, original, and hilarious tweet on the topic: "{state['topic']}".

Rules:
- DO NOT use question-answer format.
- Max 280 characters.
- Use observational humor, irony, sarcasm, or cultural references.
- Think in name logic, punchlines, or relatable takes.
- Use simple, day to day English
""")
    ]

    response = generator_llm.invoke(messages).content
    return {'tweet':response, 'tweet_history': [response]}



In [51]:

def evaluatetweet_func(state:TweetState):
    messages = [
        SystemMessage(content="You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets based on humor, originality, virality, and tweet format."),
        HumanMessage(content=f"""
Evaluate the following tweet:
                     
Tweet: "{state['tweet']}"

Use the criteria below to evaluate the tweet:
1. Originality - Is this fresh, or have you seen it a hundred times before?
2. Humor - Did it genuinely make you smile, laugh, or chuckle?
3. Punchiness - Is it short, sharp, and scroll-stopping?
4. Virality Potential - Would people retweet or share it?
5. Format - Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters)?

Auto-reject if:
- It's written in question-answer format (e.g., "Why did ..." or "What happens when ...")
- It exceeds 280 characters
- It reads like a traditional setup-punchline joke
- Don't end with generic, throwaway, or deflating lines that weaken the humor (e.g., "Masterpieces of the aubtie-uncle universe" or vague summaries)

### Respond ONLY in structured format:
- evaluation: "approved" or "needs_improvement"
- feedback: One paragraph explaining the strengths and weaknesses
""")
    ]

    response = structured_evaluator_llm.invoke(messages)
    return {'evaluation':response.evaluation, 'feedback':response.feedback,'feedback_history': [response.feedback]}

In [45]:
def optimize_tweet_func(state: TweetState):
    messages = [
        SystemMessage(content="You punch up tweets for virality and humor based on given feedback."),
        HumanMessage(content=f"""
Improve the tweet based on this feedback:
"{state['feedback']}"

Topic: "{state['topic']}"
Original Tweet: {state['tweet']}

Re-write it as a short, viral-worthy tweet. Avoid Q&A style and stay under 200 characters.
""")
    ]

    response = optimizer_llm.invoke(messages).content
    iteration = state['iteration']+1

    return {'tweet': response, 'iteration':iteration, 'tweet_history': [response]}

In [46]:
def route_evaluation(state: TweetState):
    if state['evaluation']=='approved' or state['iteration']>=state['max_iteration']:
        return 'approved'
    else:
        return 'needs_improvement'

In [47]:
graph = StateGraph(TweetState)

graph.add_node('generate', generate_tweet_func)
graph.add_node('evaluate', evaluatetweet_func)
graph.add_node('optimize', optimize_tweet_func)

graph.add_edge(START, 'generate')
graph.add_edge('generate','evaluate')
graph.add_conditional_edges('evaluate', route_evaluation, {'approved':END, 'needs_improvement':'optimize'})
graph.add_edge('optimize','evaluate')
graph.compile()

workflow = graph.compile()

In [64]:
initial_state={
    "topic": "Egdbeq",
    "iteration":1,
    "max_iteration":5
}

result = workflow.invoke(initial_state)

In [65]:
result

{'topic': 'Egdbeq',
 'tweet': 'Just discovered "Egdbeq"—the new corporate buzzword thanks to my cat’s keyboard stroll. Congrats to Fluffy, our new Senior VP of Napping! 🐾💻 #CatCareers #CorporateLife',
 'evaluation': 'approved',
 'feedback': 'This tweet showcases originality by presenting a humorous take on a corporate environment through the lens of a cat\'s accidental keyboard mashing. The playfulness of naming the cat "Senior VP of Napping" adds to the humor, and the imagery of a cat wandering on a keyboard is relatable for many pet owners. It is concise and maintains a punchy format, making it easy to read and share. The hashtags are well-chosen and enhance its potential for virality. Overall, it effectively meets all the evaluation criteria.',
 'iteration': 2,
 'max_iteration': 5,
 'tweet_history': ['Just found out that "Egdbeq" is what happens when you let a cat walk on your keyboard while trying to type an important email. Now I’m just waiting for my boss to notice the sudden pro

In [66]:
for tweet in result['tweet_history']:
    print(tweet)

Just found out that "Egdbeq" is what happens when you let a cat walk on your keyboard while trying to type an important email. Now I’m just waiting for my boss to notice the sudden promotion of "fluffy" to Senior Vice President of Napping. 🐾💻
Just discovered "Egdbeq"—the new corporate buzzword thanks to my cat’s keyboard stroll. Congrats to Fluffy, our new Senior VP of Napping! 🐾💻 #CatCareers #CorporateLife
